#Proyecto Final
##Seminario de Solucion de problemas de Inteligencia Artificial II
###Profesor: Julio Esteban Valdes Lopez
###Alumnos:
#Peña Rodriguez Carlos Alfonso
#López Arellano Ricardo David

# **Clasificador de correo SPAM**

##**Introducción**:
Clasificar correos electrónicos como `spam` o `no spam` es una tarea común en el procesamiento del lenguaje natural. Aquí te presento un ejemplo básico de cómo construir un clasificador de correo spam utilizando una red neuronal:

**Preparación de los datos:**

Recopila un conjunto de datos etiquetados que contenga correos electrónicos clasificados como spam y no spam.
Divide los datos en conjuntos de entrenamiento y prueba.

**Preprocesamiento de texto:**

Realiza una limpieza de los correos electrónicos, como eliminar signos de puntuación, convertir a minúsculas, eliminar palabras irrelevantes (como stopwords) y realizar la tokenización.
Convierte los correos electrónicos preprocesados en vectores numéricos utilizando técnicas como el modelo de bolsa de palabras (Bag-of-Words) o Word embeddings.

**Construcción del modelo de red neuronal:**

Importa las bibliotecas necesarias, como TensorFlow o Keras.
Define la arquitectura de la red neuronal, que puede incluir capas de entrada, capas ocultas y una capa de salida.
Elige la función de activación adecuada para cada capa y determina el número de neuronas en cada capa.
Compila el modelo especificando la función de pérdida y el optimizador.

**Entrenamiento del modelo:**

Ajusta el modelo utilizando los datos de entrenamiento preparados previamente.
Establece el número de épocas y el tamaño de lote (batch size) para el entrenamiento.
Observa las métricas de rendimiento, como la precisión y la pérdida, durante el entrenamiento.

**Evaluación del modelo:**

Utiliza el modelo entrenado para predecir las etiquetas de los correos electrónicos en el conjunto de prueba.
Calcula las métricas de evaluación, como la precisión, la exhaustividad y el puntaje F1, para evaluar el rendimiento del clasificador.
Recuerda que este es un enfoque básico y hay muchas formas de mejorar el rendimiento del clasificador, como la adición de capas de regularización, el ajuste de los hiperparámetros y la exploración de diferentes técnicas de preprocesamiento de texto.

In [54]:
import csv
import pandas as pd
import numpy as np

Leemos desde el dataset, ya que es una muestra

In [55]:
spam_o_NOspam = pd.read_csv("spam.csv", encoding='latin-1')[["v1", "v2"]]
spam_o_NOspam.columns = ["label", "text"]
spam_o_NOspam.head()

,label,text
0,spam,FreeMsg Hey there darling it's been 3 week's n...
1,NOspam,Even my brother is not like to speak with me. ...
2,NOspam,As per your request 'Melle Melle (Oru Minnamin...
3,spam,WINNER!! As a valued network customer you have...
4,spam,Had your mobile 11 months or more? U R entitle...


Contando cuántos son Spam y no spam

In [56]:
spam_o_NOspam["label"].value_counts()

NOspam    4821
spam       746
Name: label, dtype: int64

Con el siguiente método realizamos una optimización. Eliminación de las muestras innecesarias.

In [67]:
import string
punctuation = set(string.punctuation)
def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

Aplicamos el algoritmo de tokenización sobre la muestra con el siguiente código

In [68]:
spam_o_NOspam.head()["text"].apply(tokenize)


0    [freemsg, hey, there, darling, its, been, 3, w...
1    [even, my, brother, is, not, like, to, speak, ...
2    [as, per, your, request, melle, melle, oru, mi...
3    [winner, as, a, valued, network, customer, you...
4    [had, your, mobile, 11, months, or, more, u, r...
Name: text, dtype: object

Vamos a usar la librería scikit-learn a fin de realizar el trabajo pesado del proceso de aprendizaje y de las pruebas. Estamos diciendo qué función tiene que usar para la tokenización y que ésta debe ser binaria, es decir, no importa el número de veces que aparece una palabra, simplemente mirará si aparece o no. Con el siguiente código separamos los datos entre entrenamiento y pruebas.

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
demo_vectorizer = CountVectorizer(
    tokenizer = tokenize,
    binary = True
)

La salida de bloque es:

In [71]:
from sklearn.model_selection import train_test_split
train_text, test_text, train_labels, test_labels = train_test_split(spam_o_NOspam["text"], spam_o_NOspam["label"], stratify=spam_o_NOspam["label"])
print(f"Ejemplos de entrenamiento: {len(train_text)}, Ejemplos de prueba: {len(test_text)}")

Ejemplos de entrenamiento: 4175, Ejemplos de prueba: 1392


Nos quedaron 4175 ejemplos para entrenamiento y 1392 ejemplos para pruebas. Creamos un nuevo vectorizador, desde cero, en el que solamente vamos a usar los datos de entrenamiento, no los datos de pruebas.

In [73]:
real_vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
train_X = real_vectorizer.fit_transform(train_text)
test_X = real_vectorizer.transform(test_text)

Creamos el nuevo clasificador y usamos el método fit() para procesar los datos, lo que prepara al clasificador para usarlo más adelante. De nuevo, usamos los datos de entrenamiento para prepararlo, no los de prueba.

In [74]:
from sklearn.svm import LinearSVC
classifier = LinearSVC()
classifier.fit(train_X, train_labels)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

LinearSVC()

El clasificador está listo para trabajar con él, pero antes de ello hay que realizar una operación muy interesante que se basa en predecir la precisión de las clasificaciones que conseguirá. Para ello entra en juego otro método importante del clasificador. Se trata del método predict() que permite realizar finalmente las clasificaciones.

Para calcular la precisión usamos los datos de prueba. La función de scikit-learn llamada accuracy_score() nos sirve para calcular la puntuación de manera sencilla.

In [75]:
from sklearn.metrics import accuracy_score
predicciones = classifier.predict(test_X)
accuracy = accuracy_score(test_labels, predicciones)
print(f"Accuracy: {accuracy:.4%}")

Accuracy: 98.5632%


Insertamos las frases en un arreglo

In [76]:
frases = [
  '¿Está buscando rediseñar su sitio web con una nueva apariencia y sensación modernas?',
  'Por favor envíeme una confirmación de la eliminación completa y permanente de los datos personales',
  'Has sido seleccionado para ganar una suscripción GRATUITA a nuestro servicio',
  'Nos comunicamos con usted porque el punto final del webhook asociado con su cuenta en modo de prueba ha fallado.',
  'Confirma tu cuenta de Facebook en el siguiente link',
  'Has sido seleccionado para participar en un servicio gratuito.'
]

Las pasamos por nuestro algoritmo de transformación y vectorización, para finalmente recibir las predicciones de clasificación

In [78]:
frases_X = real_vectorizer.transform(frases)
predicciones = classifier.predict(frases_X)

Recorremos las predicciones y mostrar lo que el sistema ha sido capaz de interpretar.

In [77]:
for text, label in zip(frases, predicciones):
  print(f"{label:5} - {text}")

NOspam - ¿Está buscando rediseñar su sitio web con una nueva apariencia y sensación modernas?
spam  - Por favor envíeme una confirmación de la eliminación completa y permanente de los datos personales
NOspam - Has sido seleccionado para ganar una suscripción GRATUITA a nuestro servicio
NOspam - Nos comunicamos con usted porque el punto final del webhook asociado con su cuenta en modo de prueba ha fallado.
NOspam - Confirma tu cuenta de Facebook en el siguiente link
NOspam - Has sido seleccionado para participar en un servicio gratuito.
